In [1]:
import keras
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

/Users/viki/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/viki/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
# lets limit input to 16 digits
INPUT_DIGITS = 16
number_of_numbers = 2 ** INPUT_DIGITS - 1
input_number = []
output_label = []
for i in range(1, number_of_numbers):
    input_number.append(i)
    if i % 3 == 0 and i % 5 == 0:
        output_label.append(0) # fizzbuzz
    elif i % 3 == 0:
        output_label.append(1) # fizz
    elif i % 5 == 0:
        output_label.append(2) # buzz
    else:
        output_label.append(3) # Same

In [3]:
y = keras.utils.to_categorical(output_label)

In [4]:
def map_input_numer_to_binary(num):
    binary_rep = list("{0:b}".format(num))
    final_res = ['0' for _ in range(INPUT_DIGITS)]
    for idx, v in enumerate(reversed(binary_rep)):
        final_res[idx] = v
    return list(reversed(final_res))

In [5]:
def reverse_map_binary_to_number(binary):
    return int(''.join(binary), 2)

In [6]:
print(map_input_numer_to_binary(15), reverse_map_binary_to_number(map_input_numer_to_binary(15)))

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '1'] 15


In [7]:
x = np.array([map_input_numer_to_binary(v) for v in input_number])

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x, y, random_state = 3,)

In [9]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((49150, 16), (16384, 16), (49150, 4), (16384, 4))

In [10]:
#### define the tensorflow model

In [11]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [12]:
def model(X, w_h, w_o):
    h = tf.nn.relu(tf.matmul(X, w_h))
    return tf.matmul(h, w_o)

In [13]:
X = tf.placeholder("float", [None, INPUT_DIGITS])
Y = tf.placeholder("float", [None, 4])

NUM_HIDDEN = 100

w_h = init_weights([INPUT_DIGITS, NUM_HIDDEN]) # weight for hidden layer
w_o = init_weights([NUM_HIDDEN, 4]) # weight for output layer

py_x = model(X, w_h, w_o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=py_x, labels=Y))
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost)

predict_op = tf.argmax(py_x, 1)

BATCH_SIZE = 128

LOG_EVERY_N_EPOCHS = 100

NUMBER_OF_EPOCHS = 200

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(NUMBER_OF_EPOCHS + 1):
        permuted_indexes = np.random.permutation(range(len(x_train)))
        train_x, train_y = x_train[permuted_indexes], y_train[permuted_indexes]

        for start in range(0, len(train_x), BATCH_SIZE):
            end = start + BATCH_SIZE
            sess.run(train_op, feed_dict={X: train_x[start:end], Y: train_y[start:end]})

        if epoch % LOG_EVERY_N_EPOCHS == 0:
            print(epoch, np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict_op, feed_dict={X: train_x, Y: train_y})))

    predictions = sess.run(predict_op, feed_dict={X: x_val})

0 0.5331637843336724
100 0.7466327568667345
200 0.9378026449643947


In [15]:
def print_sample(input_value, output_prediction, expected_ouput, no_of_sampels=5, print_values=False):
    correct = 0
    count = 0
    for value, pred, expected in zip(input_value, output_prediction, expected_ouput):
        val = reverse_map_binary_to_number(value)
        pred_val = pred # np.argmax(pred)
        expected_val = np.argmax(expected)
        if print_values: print(val, pred_val, expected_val) # To prevent printing all values
        if expected_val == pred_val:
            correct += 1
        count += 1
        if no_of_sampels and count >= no_of_sampels:
            break
    print('correct', correct/count)

In [16]:
print_sample(x_val, predictions, y_val, 1000)

correct 0.939
